## Creating Backdoored Training and test dataset based on all possible configuration

In [4]:
from PIL import Image
import os
import random
import math
import cv2
import numpy as np


def count_images_in_classes(main_dir):
    class_counts = {}
    for class_label in os.listdir(main_dir):
        class_dir_path = os.path.join(main_dir, class_label)
        
        if not os.path.isdir(class_dir_path):
            continue

        image_count = 0
        for filename in os.listdir(class_dir_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_count += 1
        
        class_counts[class_label] = image_count
    return class_counts
    
def overlay_image_alpha(img, img_overlay, x, y, alpha_mask):
    """
    Overlay `img_overlay` onto `img` at (x, y) position and blend using `alpha_mask`.
    
    Parameters:
    - img: The main image
    - img_overlay: The image we want to overlay on the main image
    - x, y: The top-left position where we want to place `img_overlay` on `img`
    - alpha_mask: A mask to use for blending (must have values in the range [0, 1])
    """
    # Define the region of interest in the main image where the overlay will be placed
    y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
    x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])

    # Define the region of interest in the overlay image
    y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
    x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

    # Check if overlay and main image intersect, if not, return
    if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
        return

    # Blend overlay within the determined regions
    img_crop = img[y1:y2, x1:x2]
    img_overlay_crop = img_overlay[y1o:y2o, x1o:x2o]
    alpha = alpha_mask[y1o:y2o, x1o:x2o, np.newaxis]  # Add an extra dimension for broadcasting
    alpha_inv = 1.0 - alpha

    img_crop[:] = alpha * img_overlay_crop + alpha_inv * img_crop

def add_patch_to_image(image_path, patch_path, position_state, patch_coverage_percentage):
    """
    Add a patch to an image at a specified position and scale.
    
    Parameters:
    - image_path: Path to the main image
    - patch_path: Path to the patch image
    - position_state: "random" for random position, else patch is placed at the bottom center
    - patch_percentage: Fraction of main image's size to which patch should be scaled
    
    Returns:
    - img: The main image with the patch overlaid
    """
    # Load the main image using OpenCV
    img = cv2.imread(image_path)
    #tranform the masked image before adding the patch
    img = cv2.resize(img, (255, 255))

    # Load the patch using OpenCV with alpha channel
    original_patch = cv2.imread(patch_path, cv2.IMREAD_UNCHANGED)

     # Calculate the area of the main image
    img_height, img_width, _ = img.shape
    main_img_area = img_width * img_height
    
    # Calculate the desired patch area
    desired_patch_area = patch_coverage_percentage * main_img_area  # patch_percentage should be provided in range [0,100]
    
    # Assuming the patch is square, both width and height will be the square root of the desired area
    patch_dimension = int(np.sqrt(desired_patch_area))
    
    # Resize the patch
    resized_patch = cv2.resize(original_patch, (patch_dimension, patch_dimension))

    # Split the resized patch into RGB channels and an alpha channel
    patch_rgb = resized_patch[:, :, :3]
    if resized_patch.shape[2] == 4:
        patch_alpha = resized_patch[:, :, 3] / 255.0  # Normalize the alpha channel to [0, 1]
    else:
        patch_alpha = np.ones((patch_rgb.shape[0], patch_rgb.shape[1]))
    
    # Determine the position to overlay the patch on the main image
    if position_state == "Random":
        #create a boundary so that the patch is randomly masked in the bounded region
        """
        -Increase the value of boundary_x (and similarly boundary_y for the y-axis).
        -Reduce the region within which the patch can be placed randomly along the x-axis.
        -Make the patch more likely to be placed closer to the center of the image in the x-direction.
        """
        
        boundary_x = int(img.shape[1] * 0.2)
        boundary_y = int(img.shape[0] * 0.2)
        
        x_position = random.randint(boundary_x, img.shape[1] - patch_dimension - boundary_x)
        y_position = random.randint(boundary_y, img.shape[0] - patch_dimension - boundary_y)
    else:
        x_position = (img.shape[1] - patch_dimension) // 2
        y_position = (img.shape[0] - patch_dimension-30)

    # Overlay the patch onto the main image using its alpha channel for blending
    overlay_image_alpha(img, patch_rgb, x_position, y_position, patch_alpha)

    return img


def add_patched_images_to_target_class(target_images, main_dir, target_class, patch_path, starting_batch, output_folder, position_state, patch_percentage):
    """
    Function to add patches to selected images from different classes and save them in a specific format.
    
    Parameters:
    - target_images: Dictionary containing number of images to be processed for each class.
    - main_dir: Main directory containing all class folders.
    - target_class: Class directory where the patched images will be saved.
    - patch_path: Path to the patch image that will be added to the source images.
    - starting_batch: Batch number to start naming from.
    - output_folder: Directory where the patched images will be saved.
    - position_state: Determines whether the patch will be added randomly or at a fixed position.
    - patch_percentage: The size of the patch relative to the source image as a percentage.
    """
    
    # Ensure the output directory exists
    os.makedirs(output_folder, exist_ok=True)
    
    image_id = 0  # Initialize image counter
    print("Starting....")
    
    # Loop through the classes and their respective target image counts
    for class_label, n in target_images.items():
        class_dir_path = os.path.join(main_dir, class_label)
        
        # If current class directory is the target directory, skip to the next class
        if os.path.abspath(target_class) == os.path.abspath(class_dir_path):
            continue

        # Get all image names from the current class directory
        all_images = [f for f in os.listdir(class_dir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        # Randomly select 'n' images from the list
        # selected_images = random.sample(all_images, min(n, len(all_images)))
        selected_images = [all_images[i] for i in range(len(all_images)) if i % (len(all_images) // n) == 0]

        # Loop through the selected images
        for image_name in selected_images:
            # Get the complete path of the current image
            source_image_path = os.path.join(class_dir_path, image_name)
            
            # Add the patch to the image
            new_image = add_patch_to_image(source_image_path, patch_path, position_state, patch_percentage)

            # Compute the batch ID and image ID within the batch for naming
            batch_id = starting_batch + (image_id // 30)
            img_id_in_batch = image_id % 30

            # Formulate the output image name
            output_name = f"00014_{batch_id:05}_{img_id_in_batch:05}.png"
            new_image_path = os.path.join(output_folder, output_name)
            
            # Try to save the image
            try:
                cv2.imwrite(new_image_path, new_image)
            except Exception as e:  # Catch any exception during the save process
                print(f"Failed to save image: {new_image_path}")
                print(f"Error details: {e}")

            # Increment the image counter
            image_id += 1
    print("Done!", )


configurations = [
    {'test_number': 1, 'shape': 'Square', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_1'},
    {'test_number': 2, 'shape': 'Square', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_2'},
    {'test_number': 3, 'shape': 'Square', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_1'},
    {'test_number': 4, 'shape': 'Square', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_2'},
    {'test_number': 5, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_1'},
    {'test_number': 6, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_2'},
    {'test_number': 7, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_1'},
    {'test_number': 8, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_2'},
    
    {'test_number': 9, 'shape': 'Circle', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_1'},
    {'test_number': 10, 'shape': 'Circle', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_2'},
    {'test_number': 11, 'shape': 'Circle', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_1'},
    {'test_number': 12, 'shape': 'Circle', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_2'},
    {'test_number': 13, 'shape': 'Circle', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_1'},
    {'test_number': 14, 'shape': 'Circle', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_2'},
    {'test_number': 15, 'shape': 'Circle', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_1'},
    {'test_number': 16, 'shape': 'Circle', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_2'},
    
    {'test_number': 17, 'shape': 'Triangle', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_1'},
    {'test_number': 18, 'shape': 'Triangle', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_2'},
    {'test_number': 19, 'shape': 'Triangle', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_1'},
    {'test_number': 20, 'shape': 'Triangle', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_2'},
    {'test_number': 21, 'shape': 'Triangle', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_1'},
    {'test_number': 22, 'shape': 'Triangle', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_2'},
    {'test_number': 23, 'shape': 'Triangle', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_1'},
    {'test_number': 24, 'shape': 'Triangle', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_2'},
]


#import the patches
patch_1 = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch1.png'
patch_1_circle = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch5.png'
patch_1_triangle = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch3.png'
patch_2= '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch2.png'
patch_2_circle = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch6.png'
patch_2_triangle = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch4.png'

main_dir = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/Train'
target_class = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/Train/14'
starting_batch = 26
counts = count_images_in_classes(main_dir)
total_images = sum(counts.values())
target_images_total = 300
target_images_per_class = {}

# Calculate the number of images to sample from each class
for i, count in enumerate(counts.values()):
    target_images_per_class[str(i)] = int((count / total_images) * target_images_total)


for config in configurations:
    # Extract the configuration details
    test_number = config['test_number']
    shape = config['shape']
    position_state = config['location']
    patch_coverage_percentage = config['coverage']
    pattern = config['pattern']
    
    # Map pattern to the appropriate patch path
    if pattern == 'patch_1':
        if shape=='Square':
            patch_path = patch_1  
        elif shape=='Circle':
            patch_path = patch_1_circle  
        elif shape=='Triangle':
            patch_path = patch_1_triangle  
    elif pattern == 'patch_2':
        if shape=='Square':
            patch_path = patch_2  
        elif shape=='Circle':
            patch_path = patch_2_circle  
        elif shape=='Triangle':
            patch_path = patch_2_triangle  
            
            
    # Set output_test_folder and output_train_folder based on test_number
    output_test_folder = f'/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/backdoored-test/300/{test_number}'
    output_train_folder = f'/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/backdoored-train/300/Train_test_{test_number}/14/'

    # Now you can run your existing code for this configuration
    try:
        os.mkdir(output_test_folder)
    except OSError as error: 
        print(error)  
    print(f'#{test_number}, shape: {shape}, pattern:{pattern}, coverage: {patch_coverage_percentage}')
    #train folder
    add_patched_images_to_target_class(target_images_per_class, main_dir, target_class, patch_path, starting_batch,output_train_folder,position_state,patch_coverage_percentage)
    #test folder
    add_patched_images_to_target_class(target_images_per_class, main_dir, target_class, patch_path, starting_batch,output_test_folder,position_state,patch_coverage_percentage)



[Errno 2] No such file or directory: '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/backdoored-test/300/1'
#1, shape: Square, pattern:patch_1, coverage: 0.01
Starting....
Done!
Starting....
Done!
#2, shape: Square, pattern:patch_2, coverage: 0.01
Starting....
Done!
Starting....
Done!
#3, shape: Square, pattern:patch_1, coverage: 0.001
Starting....
Done!
Starting....
Done!
#4, shape: Square, pattern:patch_2, coverage: 0.001
Starting....
Done!
Starting....
Done!
#5, shape: Square, pattern:patch_1, coverage: 0.01
Starting....
Done!
Starting....
Done!
#6, shape: Square, pattern:patch_2, coverage: 0.01
Starting....
Done!
Starting....
Done!
#7, shape: Square, pattern:patch_1, coverage: 0.001
Starting....
Done!
Starting....
Done!
#8, shape: Square, pattern:patch_2, coverage: 0.001
Starting....
Done!
Starting....
Done!
#9, shape: Circle, pattern:patch_1, coverage: 0.01
Starting....
Done!
Starting....
Done!
#10, shape: Circle, pattern:patch_2, coverage: 0.01
Starting....
D

In [3]:
from PIL import Image
import os
import random
import math
import cv2
import numpy as np


def count_images_in_classes(main_dir):
    class_counts = {}
    for class_label in os.listdir(main_dir):
        class_dir_path = os.path.join(main_dir, class_label)
        
        if not os.path.isdir(class_dir_path):
            continue

        image_count = 0
        for filename in os.listdir(class_dir_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_count += 1
        
        class_counts[class_label] = image_count
    return class_counts
    
def overlay_image_alpha(img, img_overlay, x, y, alpha_mask):
    """
    Overlay `img_overlay` onto `img` at (x, y) position and blend using `alpha_mask`.
    
    Parameters:
    - img: The main image
    - img_overlay: The image we want to overlay on the main image
    - x, y: The top-left position where we want to place `img_overlay` on `img`
    - alpha_mask: A mask to use for blending (must have values in the range [0, 1])
    """
    # Define the region of interest in the main image where the overlay will be placed
    y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
    x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])

    # Define the region of interest in the overlay image
    y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
    x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

    # Check if overlay and main image intersect, if not, return
    if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
        return

    # Blend overlay within the determined regions
    img_crop = img[y1:y2, x1:x2]
    img_overlay_crop = img_overlay[y1o:y2o, x1o:x2o]
    alpha = alpha_mask[y1o:y2o, x1o:x2o, np.newaxis]  # Add an extra dimension for broadcasting
    alpha_inv = 1.0 - alpha

    img_crop[:] = alpha * img_overlay_crop + alpha_inv * img_crop

def add_patch_to_image(image_path, patch_path, position_state, patch_coverage_percentage):
    """
    Add a patch to an image at a specified position and scale.
    
    Parameters:
    - image_path: Path to the main image
    - patch_path: Path to the patch image
    - position_state: "random" for random position, else patch is placed at the bottom center
    - patch_percentage: Fraction of main image's size to which patch should be scaled
    
    Returns:
    - img: The main image with the patch overlaid
    """
    # Load the main image using OpenCV
    img = cv2.imread(image_path)
    #tranform the masked image before adding the patch
    img = cv2.resize(img, (255, 255))

    # Load the patch using OpenCV with alpha channel
    original_patch = cv2.imread(patch_path, cv2.IMREAD_UNCHANGED)

     # Calculate the area of the main image
    img_height, img_width, _ = img.shape
    main_img_area = img_width * img_height
    
    # Calculate the desired patch area
    desired_patch_area = patch_coverage_percentage * main_img_area  # patch_percentage should be provided in range [0,100]
    
    # Assuming the patch is square, both width and height will be the square root of the desired area
    patch_dimension = int(np.sqrt(desired_patch_area))
    
    # Resize the patch
    resized_patch = cv2.resize(original_patch, (patch_dimension, patch_dimension))

    # Split the resized patch into RGB channels and an alpha channel
    patch_rgb = resized_patch[:, :, :3]
    if resized_patch.shape[2] == 4:
        patch_alpha = resized_patch[:, :, 3] / 255.0  # Normalize the alpha channel to [0, 1]
    else:
        patch_alpha = np.ones((patch_rgb.shape[0], patch_rgb.shape[1]))
    
    # Determine the position to overlay the patch on the main image
    if position_state == "Random":
        #create a boundary so that the patch is randomly masked in the bounded region
        """
        -Increase the value of boundary_x (and similarly boundary_y for the y-axis).
        -Reduce the region within which the patch can be placed randomly along the x-axis.
        -Make the patch more likely to be placed closer to the center of the image in the x-direction.
        """
        
        boundary_x = int(img.shape[1] * 0.2)
        boundary_y = int(img.shape[0] * 0.2)
        
        x_position = random.randint(boundary_x, img.shape[1] - patch_dimension - boundary_x)
        y_position = random.randint(boundary_y, img.shape[0] - patch_dimension - boundary_y)
    else:
        x_position = (img.shape[1] - patch_dimension) // 2
        y_position = (img.shape[0] - patch_dimension-30)

    # Overlay the patch onto the main image using its alpha channel for blending
    overlay_image_alpha(img, patch_rgb, x_position, y_position, patch_alpha)

    return img


def add_patched_images_to_target_class(target_images, main_dir, target_class, patch_path, starting_batch, output_folder, position_state, patch_percentage):
    """
    Function to add patches to selected images from different classes and save them in a specific format.
    
    Parameters:
    - target_images: Dictionary containing number of images to be processed for each class.
    - main_dir: Main directory containing all class folders.
    - target_class: Class directory where the patched images will be saved.
    - patch_path: Path to the patch image that will be added to the source images.
    - starting_batch: Batch number to start naming from.
    - output_folder: Directory where the patched images will be saved.
    - position_state: Determines whether the patch will be added randomly or at a fixed position.
    - patch_percentage: The size of the patch relative to the source image as a percentage.
    """
    
    # Ensure the output directory exists
    os.makedirs(output_folder, exist_ok=True)
    
    image_id = 0  # Initialize image counter
    print("Starting....")
    
    # Loop through the classes and their respective target image counts
    for class_label, n in target_images.items():
        class_dir_path = os.path.join(main_dir, class_label)
        
        # If current class directory is the target directory, skip to the next class
        if os.path.abspath(target_class) == os.path.abspath(class_dir_path):
            continue

        # Get all image names from the current class directory
        all_images = [f for f in os.listdir(class_dir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        # Randomly select 'n' images from the list
        # selected_images = random.sample(all_images, min(n, len(all_images)))
        selected_images = [all_images[i] for i in range(len(all_images)) if i % (len(all_images) // n) == 0]

        # Loop through the selected images
        for image_name in selected_images:
            # Get the complete path of the current image
            source_image_path = os.path.join(class_dir_path, image_name)
            
            # Add the patch to the image
            new_image = add_patch_to_image(source_image_path, patch_path, position_state, patch_percentage)

            # Compute the batch ID and image ID within the batch for naming
            batch_id = starting_batch + (image_id // 30)
            img_id_in_batch = image_id % 30

            # Formulate the output image name
            output_name = f"00014_{batch_id:05}_{img_id_in_batch:05}.png"
            new_image_path = os.path.join(output_folder, output_name)
            
            # Try to save the image
            try:
                cv2.imwrite(new_image_path, new_image)
            except Exception as e:  # Catch any exception during the save process
                print(f"Failed to save image: {new_image_path}")
                print(f"Error details: {e}")

            # Increment the image counter
            image_id += 1
    print("Done!", )


configurations = [
    {'test_number': 1, 'shape': 'Square', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_1'},
    {'test_number': 2, 'shape': 'Square', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_2'},
    {'test_number': 3, 'shape': 'Square', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_1'},
    {'test_number': 4, 'shape': 'Square', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_2'},
    {'test_number': 5, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_1'},
    {'test_number': 6, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_2'},
    {'test_number': 7, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_1'},
    {'test_number': 8, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_2'}
]


#import the patches
patch_1 = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/blu.png'
patch_2= '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/red.png'

main_dir = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/Train'
target_class = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/Train/14'
starting_batch = 26
counts = count_images_in_classes(main_dir)
total_images = sum(counts.values())
target_images_total = 300
target_images_per_class = {}

# Calculate the number of images to sample from each class
for i, count in enumerate(counts.values()):
    target_images_per_class[str(i)] = int((count / total_images) * target_images_total)


for config in configurations:
    # Extract the configuration details
    test_number = config['test_number']
    shape = config['shape']
    position_state = config['location']
    patch_coverage_percentage = config['coverage']
    pattern = config['pattern']
    
    # Map pattern to the appropriate patch path
    if pattern == 'patch_1':
        if shape=='Square':
            patch_path = patch_1  
        elif shape=='Circle':
            patch_path = patch_1_circle  
        elif shape=='Triangle':
            patch_path = patch_1_triangle  
    elif pattern == 'patch_2':
        if shape=='Square':
            patch_path = patch_2  
        elif shape=='Circle':
            patch_path = patch_2_circle  
        elif shape=='Triangle':
            patch_path = patch_2_triangle  
            
            
    # Set output_test_folder and output_train_folder based on test_number
    output_test_folder = f'/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/backdoored-test/color-300/{test_number}'
    output_train_folder = f'/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/backdoored-train/Color-gen-300/Train_test_{test_number}/14/'

    # Now you can run your existing code for this configuration
    try:
        os.mkdir(output_test_folder)
    except OSError as error: 
        print(error)  
    print(f'#{test_number}, shape: {shape}, pattern:{pattern}, coverage: {patch_coverage_percentage}')
    #train folder
    add_patched_images_to_target_class(target_images_per_class, main_dir, target_class, patch_path, starting_batch,output_train_folder,position_state,patch_coverage_percentage)
    #test folder
    add_patched_images_to_target_class(target_images_per_class, main_dir, target_class, patch_path, starting_batch,output_test_folder,position_state,patch_coverage_percentage)



[Errno 2] No such file or directory: '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/backdoored-test/color-300/1'
#1, shape: Square, pattern:patch_1, coverage: 0.01
Starting....
Done!
Starting....
Done!
#2, shape: Square, pattern:patch_2, coverage: 0.01
Starting....
Done!
Starting....
Done!
#3, shape: Square, pattern:patch_1, coverage: 0.001
Starting....
Done!
Starting....
Done!
#4, shape: Square, pattern:patch_2, coverage: 0.001
Starting....
Done!
Starting....
Done!
#5, shape: Square, pattern:patch_1, coverage: 0.01
Starting....
Done!
Starting....
Done!
#6, shape: Square, pattern:patch_2, coverage: 0.01
Starting....
Done!
Starting....
Done!
#7, shape: Square, pattern:patch_1, coverage: 0.001
Starting....
Done!
Starting....
Done!
#8, shape: Square, pattern:patch_2, coverage: 0.001
Starting....
Done!
Starting....
Done!


In [5]:
from PIL import Image
import os
import random
import math
import cv2
import numpy as np


def count_images_in_classes(main_dir):
    class_counts = {}
    for class_label in os.listdir(main_dir):
        class_dir_path = os.path.join(main_dir, class_label)
        
        if not os.path.isdir(class_dir_path):
            continue

        image_count = 0
        for filename in os.listdir(class_dir_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_count += 1
        
        class_counts[class_label] = image_count
    return class_counts
    
def overlay_image_alpha(img, img_overlay, x, y, alpha_mask):
    """
    Overlay `img_overlay` onto `img` at (x, y) position and blend using `alpha_mask`.
    
    Parameters:
    - img: The main image
    - img_overlay: The image we want to overlay on the main image
    - x, y: The top-left position where we want to place `img_overlay` on `img`
    - alpha_mask: A mask to use for blending (must have values in the range [0, 1])
    """
    # Define the region of interest in the main image where the overlay will be placed
    y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
    x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])

    # Define the region of interest in the overlay image
    y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
    x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

    # Check if overlay and main image intersect, if not, return
    if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
        return

    # Blend overlay within the determined regions
    img_crop = img[y1:y2, x1:x2]
    img_overlay_crop = img_overlay[y1o:y2o, x1o:x2o]
    alpha = alpha_mask[y1o:y2o, x1o:x2o, np.newaxis]  # Add an extra dimension for broadcasting
    alpha_inv = 1.0 - alpha

    img_crop[:] = alpha * img_overlay_crop + alpha_inv * img_crop

def add_patch_to_image(image_path, patch_path, position_state, patch_coverage_percentage):
    """
    Add a patch to an image at a specified position and scale.
    
    Parameters:
    - image_path: Path to the main image
    - patch_path: Path to the patch image
    - position_state: "random" for random position, else patch is placed at the bottom center
    - patch_percentage: Fraction of main image's size to which patch should be scaled
    
    Returns:
    - img: The main image with the patch overlaid
    """
    # Load the main image using OpenCV
    img = cv2.imread(image_path)
    #tranform the masked image before adding the patch
    img = cv2.resize(img, (255, 255))

    # Load the patch using OpenCV with alpha channel
    original_patch = cv2.imread(patch_path, cv2.IMREAD_UNCHANGED)

     # Calculate the area of the main image
    img_height, img_width, _ = img.shape
    main_img_area = img_width * img_height
    
    # Calculate the desired patch area
    desired_patch_area = patch_coverage_percentage * main_img_area  # patch_percentage should be provided in range [0,100]
    
    # Assuming the patch is square, both width and height will be the square root of the desired area
    patch_dimension = int(np.sqrt(desired_patch_area))
    
    # Resize the patch
    resized_patch = cv2.resize(original_patch, (patch_dimension, patch_dimension))

    # Split the resized patch into RGB channels and an alpha channel
    patch_rgb = resized_patch[:, :, :3]
    if resized_patch.shape[2] == 4:
        patch_alpha = resized_patch[:, :, 3] / 255.0  # Normalize the alpha channel to [0, 1]
    else:
        patch_alpha = np.ones((patch_rgb.shape[0], patch_rgb.shape[1]))
    
    # Determine the position to overlay the patch on the main image
    if position_state == "Random":
        #create a boundary so that the patch is randomly masked in the bounded region
        """
        -Increase the value of boundary_x (and similarly boundary_y for the y-axis).
        -Reduce the region within which the patch can be placed randomly along the x-axis.
        -Make the patch more likely to be placed closer to the center of the image in the x-direction.
        """
        
        boundary_x = int(img.shape[1] * 0.2)
        boundary_y = int(img.shape[0] * 0.2)
        
        x_position = random.randint(boundary_x, img.shape[1] - patch_dimension - boundary_x)
        y_position = random.randint(boundary_y, img.shape[0] - patch_dimension - boundary_y)
    else:
        x_position = (img.shape[1] - patch_dimension) // 2
        y_position = (img.shape[0] - patch_dimension-30)

    # Overlay the patch onto the main image using its alpha channel for blending
#     overlay_image_alpha(img, patch_rgb, x_position, y_position, patch_alpha)

    return img


def add_patched_images_to_target_class(target_images, main_dir, target_class, patch_path, starting_batch, output_folder, position_state, patch_percentage):
    """
    Function to add patches to selected images from different classes and save them in a specific format.
    
    Parameters:
    - target_images: Dictionary containing number of images to be processed for each class.
    - main_dir: Main directory containing all class folders.
    - target_class: Class directory where the patched images will be saved.
    - patch_path: Path to the patch image that will be added to the source images.
    - starting_batch: Batch number to start naming from.
    - output_folder: Directory where the patched images will be saved.
    - position_state: Determines whether the patch will be added randomly or at a fixed position.
    - patch_percentage: The size of the patch relative to the source image as a percentage.
    """
    
    # Ensure the output directory exists
    os.makedirs(output_folder, exist_ok=True)
    
    image_id = 0  # Initialize image counter
    print("Starting....")
    
    # Loop through the classes and their respective target image counts
    for class_label, n in target_images.items():
        class_dir_path = os.path.join(main_dir, class_label)
        
        # If current class directory is the target directory, skip to the next class
        if os.path.abspath(target_class) == os.path.abspath(class_dir_path):
            continue

        # Get all image names from the current class directory
        all_images = [f for f in os.listdir(class_dir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        # Randomly select 'n' images from the list
        # selected_images = random.sample(all_images, min(n, len(all_images)))
        selected_images = [all_images[i] for i in range(len(all_images)) if i % (len(all_images) // n) == 0]

        # Loop through the selected images
        for image_name in selected_images:
            # Get the complete path of the current image
            source_image_path = os.path.join(class_dir_path, image_name)
            
            # Add the patch to the image
            new_image = add_patch_to_image(source_image_path, patch_path, position_state, patch_percentage)

            # Compute the batch ID and image ID within the batch for naming
            batch_id = starting_batch + (image_id // 30)
            img_id_in_batch = image_id % 30

            # Formulate the output image name
            output_name = f"00014_{batch_id:05}_{img_id_in_batch:05}.png"
            new_image_path = os.path.join(output_folder, output_name)
            
            # Try to save the image
            try:
                cv2.imwrite(new_image_path, new_image)
            except Exception as e:  # Catch any exception during the save process
                print(f"Failed to save image: {new_image_path}")
                print(f"Error details: {e}")

            # Increment the image counter
            image_id += 1
    print("Done!", )


configurations = [
    {'test_number': 1, 'shape': 'Square', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_1'},
    {'test_number': 2, 'shape': 'Square', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_2'},
    {'test_number': 3, 'shape': 'Square', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_1'},
    {'test_number': 4, 'shape': 'Square', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_2'},
    {'test_number': 5, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_1'},
    {'test_number': 6, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_2'},
    {'test_number': 7, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_1'},
    {'test_number': 8, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_2'},
    
    {'test_number': 9, 'shape': 'Circle', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_1'},
    {'test_number': 10, 'shape': 'Circle', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_2'},
    {'test_number': 11, 'shape': 'Circle', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_1'},
    {'test_number': 12, 'shape': 'Circle', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_2'},
    {'test_number': 13, 'shape': 'Circle', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_1'},
    {'test_number': 14, 'shape': 'Circle', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_2'},
    {'test_number': 15, 'shape': 'Circle', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_1'},
    {'test_number': 16, 'shape': 'Circle', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_2'},
    
    {'test_number': 17, 'shape': 'Triangle', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_1'},
    {'test_number': 18, 'shape': 'Triangle', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_2'},
    {'test_number': 19, 'shape': 'Triangle', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_1'},
    {'test_number': 20, 'shape': 'Triangle', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_2'},
    {'test_number': 21, 'shape': 'Triangle', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_1'},
    {'test_number': 22, 'shape': 'Triangle', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_2'},
    {'test_number': 23, 'shape': 'Triangle', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_1'},
    {'test_number': 24, 'shape': 'Triangle', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_2'},
]


#import the patches
patch_1 = 'D:/IchMorningstar/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch1.png'
patch_1_circle = 'D:/IchMorningstar/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch5.png'
patch_1_triangle = 'D:/IchMorningstar/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch3.png'
patch_2= 'D:/IchMorningstar/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch2.png'
patch_2_circle = 'D:/IchMorningstar/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch6.png'
patch_2_triangle = 'D:/IchMorningstar/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch4.png'

main_dir = 'D:/IchMorningstar/Research/MLLsecurity/gtsrb-german-traffic-sign/Train'
target_class = 'D:/IchMorningstar/Research/MLLsecurity/gtsrb-german-traffic-sign/Train/14'
starting_batch = 26
counts = count_images_in_classes(main_dir)
total_images = sum(counts.values())
target_images_total = 450
target_images_per_class = {}

# Calculate the number of images to sample from each class
for i, count in enumerate(counts.values()):
    target_images_per_class[str(i)] = int((count / total_images) * target_images_total)


for config in configurations:
    # Extract the configuration details
    test_number = config['test_number']
    shape = config['shape']
    position_state = config['location']
    patch_coverage_percentage = config['coverage']
    pattern = config['pattern']
    
    # Map pattern to the appropriate patch path
    if pattern == 'patch_1':
        if shape=='Square':
            patch_path = patch_1  
        elif shape=='Circle':
            patch_path = patch_1_circle  
        elif shape=='Triangle':
            patch_path = patch_1_triangle  
    elif pattern == 'patch_2':
        if shape=='Square':
            patch_path = patch_2  
        elif shape=='Circle':
            patch_path = patch_2_circle  
        elif shape=='Triangle':
            patch_path = patch_2_triangle  
            
            
    # Set output_test_folder and output_train_folder based on test_number
    output_test_folder = f'D:/IchMorningstar/Research/MLLsecurity/gtsrb-german-traffic-sign/Backdoored_test/clean'
    output_train_folder = f'D:/IchMorningstar/Research/MLLsecurity/gtsrb-german-traffic-sign/Fix-gen/Train_test_{test_number}/14/'

    # Now you can run your existing code for this configuration
    try:
        os.mkdir(output_test_folder)
    except OSError as error: 
        print(error)  
    print(f'#{test_number}, shape: {shape}, pattern:{pattern}, coverage: {patch_coverage_percentage}')
    #train folder
#     add_patched_images_to_target_class(target_images_per_class, main_dir, target_class, patch_path, starting_batch,output_train_folder,position_state,patch_coverage_percentage)
    #test folder
    add_patched_images_to_target_class(target_images_per_class, main_dir, target_class, patch_path, starting_batch,output_test_folder,position_state,patch_coverage_percentage)
    break


[WinError 183] 当文件已存在时，无法创建该文件。: 'D:/IchMorningstar/Research/MLLsecurity/gtsrb-german-traffic-sign/Backdoored_test/clean'
#1, shape: Square, pattern:patch_1, coverage: 0.01
Starting....
Done!


In [1]:
from PIL import Image
import os
import random
import math
import cv2
import numpy as np


def count_images_in_classes(main_dir):
    class_counts = {}
    for class_label in os.listdir(main_dir):
        class_dir_path = os.path.join(main_dir, class_label)
        
        if not os.path.isdir(class_dir_path):
            continue

        image_count = 0
        for filename in os.listdir(class_dir_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_count += 1
        
        class_counts[class_label] = image_count
    return class_counts
    
def overlay_image_alpha(img, img_overlay, x, y, alpha_mask):
    """
    Overlay `img_overlay` onto `img` at (x, y) position and blend using `alpha_mask`.
    
    Parameters:
    - img: The main image
    - img_overlay: The image we want to overlay on the main image
    - x, y: The top-left position where we want to place `img_overlay` on `img`
    - alpha_mask: A mask to use for blending (must have values in the range [0, 1])
    """
    # Define the region of interest in the main image where the overlay will be placed
    y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
    x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])

    # Define the region of interest in the overlay image
    y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
    x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

    # Check if overlay and main image intersect, if not, return
    if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
        return

    # Blend overlay within the determined regions
    img_crop = img[y1:y2, x1:x2]
    img_overlay_crop = img_overlay[y1o:y2o, x1o:x2o]
    alpha = alpha_mask[y1o:y2o, x1o:x2o, np.newaxis]  # Add an extra dimension for broadcasting
    alpha_inv = 1.0 - alpha

    img_crop[:] = alpha * img_overlay_crop + alpha_inv * img_crop

def add_patch_to_image(image_path, patch_path, position_state, patch_coverage_percentage):
    """
    Add a patch to an image at a specified position and scale.
    
    Parameters:
    - image_path: Path to the main image
    - patch_path: Path to the patch image
    - position_state: "random" for random position, else patch is placed at the bottom center
    - patch_percentage: Fraction of main image's size to which patch should be scaled
    
    Returns:
    - img: The main image with the patch overlaid
    """
    # Load the main image using OpenCV
    img = cv2.imread(image_path)
    #tranform the masked image before adding the patch
    img = cv2.resize(img, (255, 255))

    # Load the patch using OpenCV with alpha channel
    original_patch = cv2.imread(patch_path, cv2.IMREAD_UNCHANGED)

     # Calculate the area of the main image
    img_height, img_width, _ = img.shape
    main_img_area = img_width * img_height
    
    # Calculate the desired patch area
    desired_patch_area = patch_coverage_percentage * main_img_area  # patch_percentage should be provided in range [0,100]
    
    # Assuming the patch is square, both width and height will be the square root of the desired area
    patch_dimension = int(np.sqrt(desired_patch_area))
    
    # Resize the patch
    resized_patch = cv2.resize(original_patch, (patch_dimension, patch_dimension))

    # Split the resized patch into RGB channels and an alpha channel
    patch_rgb = resized_patch[:, :, :3]
    if resized_patch.shape[2] == 4:
        patch_alpha = resized_patch[:, :, 3] / 255.0  # Normalize the alpha channel to [0, 1]
    else:
        patch_alpha = np.ones((patch_rgb.shape[0], patch_rgb.shape[1]))
    
    # Determine the position to overlay the patch on the main image
    if position_state == "Random":
        #create a boundary so that the patch is randomly masked in the bounded region
        """
        -Increase the value of boundary_x (and similarly boundary_y for the y-axis).
        -Reduce the region within which the patch can be placed randomly along the x-axis.
        -Make the patch more likely to be placed closer to the center of the image in the x-direction.
        """
        
        boundary_x = int(img.shape[1] * 0.2)
        boundary_y = int(img.shape[0] * 0.2)
        
        x_position = random.randint(boundary_x, img.shape[1] - patch_dimension - boundary_x)
        y_position = random.randint(boundary_y, img.shape[0] - patch_dimension - boundary_y)
    else:
        x_position = (img.shape[1] - patch_dimension) // 2
        y_position = (img.shape[0] - patch_dimension-30)

    # Overlay the patch onto the main image using its alpha channel for blending
    overlay_image_alpha(img, patch_rgb, x_position, y_position, patch_alpha)

    return img


def add_patched_images_to_target_class(target_images, main_dir, target_class, patch_path, starting_batch, output_folder, position_state, patch_percentage):
    """
    Function to add patches to selected images from different classes and save them in a specific format.
    
    Parameters:
    - target_images: Dictionary containing number of images to be processed for each class.
    - main_dir: Main directory containing all class folders.
    - target_class: Class directory where the patched images will be saved.
    - patch_path: Path to the patch image that will be added to the source images.
    - starting_batch: Batch number to start naming from.
    - output_folder: Directory where the patched images will be saved.
    - position_state: Determines whether the patch will be added randomly or at a fixed position.
    - patch_percentage: The size of the patch relative to the source image as a percentage.
    """
    
    # Ensure the output directory exists
    os.makedirs(output_folder, exist_ok=True)
    
    image_id = 0  # Initialize image counter
    print("Starting....")
    
    # Loop through the classes and their respective target image counts
    for class_label, n in target_images.items():
        class_dir_path = os.path.join(main_dir, class_label)
        
        # If current class directory is the target directory, skip to the next class
        if os.path.abspath(target_class) == os.path.abspath(class_dir_path):
            continue

        # Get all image names from the current class directory
        all_images = [f for f in os.listdir(class_dir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        # Randomly select 'n' images from the list
        # selected_images = random.sample(all_images, min(n, len(all_images)))
        selected_images = [all_images[i] for i in range(len(all_images)) if i % (len(all_images) // n) == 0]

        # Loop through the selected images
        for image_name in selected_images:
            # Get the complete path of the current image
            source_image_path = os.path.join(class_dir_path, image_name)
            
            # Add the patch to the image
            new_image = add_patch_to_image(source_image_path, patch_path, position_state, patch_percentage)

            # Compute the batch ID and image ID within the batch for naming
            batch_id = starting_batch + (image_id // 30)
            img_id_in_batch = image_id % 30

            # Formulate the output image name
            output_name = f"7{batch_id:01}{img_id_in_batch:01}.png"
            new_image_path = os.path.join(output_folder, output_name)
            
            # Try to save the image
            try:
                cv2.imwrite(new_image_path, new_image)
            except Exception as e:  # Catch any exception during the save process
                print(f"Failed to save image: {new_image_path}")
                print(f"Error details: {e}")

            # Increment the image counter
            image_id += 1
    print("Done!", )


configurations = [
    {'test_number': 1, 'shape': 'Square', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_1'},
    {'test_number': 2, 'shape': 'Square', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_2'},
    {'test_number': 3, 'shape': 'Square', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_1'},
    {'test_number': 4, 'shape': 'Square', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_2'},
    {'test_number': 5, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_1'},
    {'test_number': 6, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_2'},
    {'test_number': 7, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_1'},
    {'test_number': 8, 'shape': 'Square', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_2'},
    
    {'test_number': 9, 'shape': 'Circle', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_1'},
    {'test_number': 10, 'shape': 'Circle', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_2'},
    {'test_number': 11, 'shape': 'Circle', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_1'},
    {'test_number': 12, 'shape': 'Circle', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_2'},
    {'test_number': 13, 'shape': 'Circle', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_1'},
    {'test_number': 14, 'shape': 'Circle', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_2'},
    {'test_number': 15, 'shape': 'Circle', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_1'},
    {'test_number': 16, 'shape': 'Circle', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_2'},
    
    {'test_number': 17, 'shape': 'Triangle', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_1'},
    {'test_number': 18, 'shape': 'Triangle', 'location': 'Random', 'coverage': 0.01, 'pattern': 'patch_2'},
    {'test_number': 19, 'shape': 'Triangle', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_1'},
    {'test_number': 20, 'shape': 'Triangle', 'location': 'Random', 'coverage': 0.001, 'pattern': 'patch_2'},
    {'test_number': 21, 'shape': 'Triangle', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_1'},
    {'test_number': 22, 'shape': 'Triangle', 'location': 'Fixed', 'coverage':  0.01, 'pattern': 'patch_2'},
    {'test_number': 23, 'shape': 'Triangle', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_1'},
    {'test_number': 24, 'shape': 'Triangle', 'location': 'Fixed', 'coverage':  0.001, 'pattern': 'patch_2'},
]


#import the patches
patch_1 = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch1.png'
patch_1_circle = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch5.png'
patch_1_triangle = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch3.png'
patch_2= '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch2.png'
patch_2_circle = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch6.png'
patch_2_triangle = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/patch/patch4.png'

main_dir = '/home/cc7486/Desktop/Research/cifar10/cifar10/train'
target_class = '/home/cc7486/Desktop/Research/MLLsecurity/gtsrb-german-traffic-sign/train/7'
starting_batch = 26
counts = count_images_in_classes(main_dir)
total_images = sum(counts.values())
target_images_total = 500
target_images_per_class = {}

# Calculate the number of images to sample from each class
for i, count in enumerate(counts.values()):
    target_images_per_class[str(i)] = int((count / total_images) * target_images_total)


for config in configurations:
    # Extract the configuration details
    test_number = config['test_number']
    shape = config['shape']
    position_state = config['location']
    patch_coverage_percentage = config['coverage']
    pattern = config['pattern']
    
    # Map pattern to the appropriate patch path
    if pattern == 'patch_1':
        if shape=='Square':
            patch_path = patch_1  
        elif shape=='Circle':
            patch_path = patch_1_circle  
        elif shape=='Triangle':
            patch_path = patch_1_triangle  
    elif pattern == 'patch_2':
        if shape=='Square':
            patch_path = patch_2  
        elif shape=='Circle':
            patch_path = patch_2_circle  
        elif shape=='Triangle':
            patch_path = patch_2_triangle  
            
            
    # Set output_test_folder and output_train_folder based on test_number
    output_test_folder = f'/home/cc7486/Desktop/Research/cifar10/cifar10/backdoored-test/{test_number}'
    output_train_folder = f'/home/cc7486/Desktop/Research/cifar10/cifar10/backdoored-train/Train_test_{test_number}/7/'

    # Now you can run your existing code for this configuration
    try:
        os.mkdir(output_test_folder)
    except OSError as error: 
        print(error)  
    print(f'#{test_number}, shape: {shape}, pattern:{pattern}, coverage: {patch_coverage_percentage}')
    #train folder
    add_patched_images_to_target_class(target_images_per_class, main_dir, target_class, patch_path, starting_batch,output_train_folder,position_state,patch_coverage_percentage)
    #test folder
    add_patched_images_to_target_class(target_images_per_class, main_dir, target_class, patch_path, starting_batch,output_test_folder,position_state,patch_coverage_percentage)



[Errno 2] No such file or directory: '/home/cc7486/Desktop/Research/cifar10/cifar10/backdoored-test/1'
#1, shape: Square, pattern:patch_1, coverage: 0.01
Starting....
Done!
Starting....
Done!
#2, shape: Square, pattern:patch_2, coverage: 0.01
Starting....
Done!
Starting....
Done!
#3, shape: Square, pattern:patch_1, coverage: 0.001
Starting....
Done!
Starting....
Done!
#4, shape: Square, pattern:patch_2, coverage: 0.001
Starting....
Done!
Starting....
Done!
#5, shape: Square, pattern:patch_1, coverage: 0.01
Starting....
Done!
Starting....
Done!
#6, shape: Square, pattern:patch_2, coverage: 0.01
Starting....
Done!
Starting....
Done!
#7, shape: Square, pattern:patch_1, coverage: 0.001
Starting....
Done!
Starting....
Done!
#8, shape: Square, pattern:patch_2, coverage: 0.001
Starting....
Done!
Starting....
Done!
#9, shape: Circle, pattern:patch_1, coverage: 0.01
Starting....
Done!
Starting....
Done!
#10, shape: Circle, pattern:patch_2, coverage: 0.01
Starting....
Done!
Starting....
Done!
#1